This is a testing notebook for processing IRE1 cluster data based on CSV output files from Cell Profiler.

In [ ]:
# load modules

# uncomment for debugging
%load_ext autoreload
%autoreload 2
%matplotlib

import os, sys, inspect
import matplotlib.pylab as plt
import numpy as np
import pandas as pd
import pprint
import re
import time
import seaborn as sns
import warnings


# Disable future warnings for seaborn
warnings.simplefilter(action='ignore', category=FutureWarning)


# Add source code directory (src) to path to enable module import
curr_frame = inspect.getfile(inspect.currentframe())
curr_dir = os.path.dirname(os.path.abspath(curr_frame))
parent_dir = os.path.dirname(curr_dir)
module_dir = os.path.join(parent_dir, 'src')
os.sys.path.insert(0, module_dir)

import cellprofiler_tools as cpt


In [ ]:
# Load data from CSV files
data_dir = '../data/processed/CellProfiler_FociQuant03/csv_outputs'

image_file_csv = 'FociQuant03_Image.csv'
er_masks_csv = 'FociQuant03_ER_masks_accepted.csv'
ire1_clust_csv = 'FociQuant03_IRE1_clusters.csv'


# Load the image file
image_full_file = os.path.join(data_dir, image_file_csv)
images = cpt.get_data_cp_csv(image_full_file)
#images = cpt.get_data_cp_csv(image_full_file, data_fields=['ImageNumber','FileName_DNA_DAPI'])

er_masks = cpt.get_data_cp_csv(os.path.join(data_dir, er_masks_csv))
ire1_clust = cpt.get_data_cp_csv(os.path.join(data_dir, ire1_clust_csv))

print('Loaded')

In [ ]:
# Plot fraction of cells with clusters per condition
condition = 'Metadata_hours_Tm'

# Create a dataframe for all cells that are included in the analysis
cells = er_masks.copy()
cells.index.name = 'Cell_ID'

cpt.add_image_prop_to_cells (cells, images, condition)

cells['Has_IRE1_clusters'] = cells['Children_IRE1_clusters_Count'].astype('bool')

frac_clust = cpt.bootstrap_cell_prop (cells, 'Has_IRE1_clusters', condition)

ax = sns.barplot(data=frac_clust, color='steelblue', ci="sd")
ax.set_title('Formation and dissolution of IRE1 clusters in stressed cells')
ax.set_xlabel('Hours of Tm treatment')
ax.set_ylabel('Fraction of cells with clusters')
ax.set_ylim(bottom=0)
plt.show()

In [ ]:
# Plot cluster properties per cell

#prop = 'AreaShape_Compactness'
#prop = 'AreaShape_Area'
prop = 'Intensity_IntegratedIntensity_IRE1_mNeonGreen'
stat='sum'

result_name = 'IRE1_clust_'+prop+'_'+stat
rel_col = 'Parent_ER_masks_accepted'
condition = 'Metadata_hours_Tm'


cpt.add_child_prop_to_cells (cells, ire1_clust, prop, rel_col, 
                             result_name, statistic=stat)
cells_valid = cells.dropna()

result = cpt.bootstrap_cell_prop (cells_valid, result_name, condition)


ax = sns.barplot(data=result, color='steelblue', ci="sd")
ax.set_title(result_name)
ax.set_xlabel(condition)
ax.set_ylabel(result_name)
ax.set_ylim(bottom=0)
plt.show()

In [ ]:
# Plot number of cluster per cell for cells that have clusters

result_name = 'Children_IRE1_clusters_Count'
condition = 'Metadata_hours_Tm'

cells_valid = cells.dropna()

result = cpt.bootstrap_cell_prop (cells_valid, result_name, condition)

ax = sns.barplot(data=result, color='steelblue', ci="sd")
ax.set_title(result_name)
ax.set_xlabel(condition)
ax.set_ylabel(result_name)
ax.set_ylim(bottom=0)
plt.show()



In [ ]:
"""
# Uncomment for a quick timer
start = time.time()
print("Start timer")

end = time.time()
print(end - start)
"""

string = 'C1-AVG_vVB_190416_03_Well03_02hTm_1_MMStack_Pos0.ome.tif'


z = re.match(r".*_(\d+)hTm_.*", string)

print(z.group(1))

